In [45]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import requests
import json
import pickle

In [39]:
df = pd.read_csv("Gbif Total.csv", delimiter="\t", low_memory=False)

In [40]:
geometry = [Point(xy) for xy in zip (df['decimalLongitude'], df['decimalLatitude'])]

In [41]:
k = df['datasetKey'].unique()
n = ([])
for key in k:
    url = "http://api.gbif.org/v1/dataset/" + key
    name = requests.get(url).json()['title']
    n.append(name)
conv = dict(zip(k,n))  
print(conv)

{'be5507b9-7abf-4b69-afe1-5ca2b7561734': u'University of Michigan Museum of Zoology, Division of Birds', '51c49096-dc31-4077-b35c-4510a8ee1ee8': u'NCSM Mollusks Collection', 'cece4fc2-1fec-4bb5-a335-7252548e3f0b': u'CAS Herpetology (HERP)', '262f8270-f9c2-4bc6-a562-8ed71c0790e6': u'Stuart M. Fullerton Collection of Arthropods (UCFC), University of Central Florida', 'b281d0c7-e700-4067-a47f-39eafda1b893': u'Geographic distribution of dinoflagellate cysts in surface sediments', '7827f68d-c981-4023-bace-288a03434044': u'Intermountain Herbarium (Utah State University)', '41fc5c40-5e81-496f-9733-6b5681b3b7a5': u'Field Museum of Natural History (Zoology) Mammal Collection', '8e02e1dd-ec54-405e-b4d7-7abdecd29cc7': u'University of Minnesota Insect Collection', '96ce10e8-f762-11e1-a439-00145eb45e9a': u'Lichen Collection - University of Washington Herbarium (WTU)', '7adf20e0-c955-11de-95c0-b8a03c50a862': u'Royal Botanic Garden Edinburgh Living Plant Collections (E)', 'c8681cc2-9d0a-4c5f-b620-5c7

In [42]:
df['datasetName'] = df['datasetKey']
df.replace({'datasetName': conv})

,gbifID,datasetKey,occurrenceID,kingdom,phylum,class,order,family,genus,species,...,dateIdentified,license,rightsHolder,recordedBy,typeStatus,establishmentMeans,lastInterpreted,mediaType,issue,datasetName
0,1900971445,4bfac3ea-8763-4f4b-a71a-76a6f5f243d3,MCZ:IZ:150509,Animalia,Arthropoda,Arachnida,Araneae,Linyphiidae,Neriene,Neriene digna,...,NaN,CC_BY_NC_4_0,President and Fellows of Harvard College,James H. Emerton Collection,NaN,NaN,2019-04-11T19:32:32.459Z,NaN,COORDINATE_ROUNDED;GEODETIC_DATUM_INVALID;GEOD...,"Museum of Comparative Zoology, Harvard University"
1,1901026404,50c9509d-22c7-4a22-a47d-8c48425ef4a7,https://www.inaturalist.org/observations/11531191,Plantae,Tracheophyta,Magnoliopsida,Asterales,Asteraceae,Tanacetum,Tanacetum vulgare,...,NaN,CC_BY_NC_4_0,jaybirdclarkson,jaybirdclarkson,NaN,NaN,2019-03-19T19:22:41.231Z,STILLIMAGE,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS84,iNaturalist Research-grade Observations
2,1901026536,50c9509d-22c7-4a22-a47d-8c48425ef4a7,https://www.inaturalist.org/observations/11772837,Plantae,Tracheophyta,Polypodiopsida,Polypodiales,Polypodiaceae,Polypodium,Polypodium glycyrrhiza,...,2018-04-29T18:32:01Z,CC_BY_NC_4_0,Melissa,Melissa,NaN,NaN,2019-03-19T19:24:23.416Z,NaN,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS84,iNaturalist Research-grade Observations
3,1901027735,50c9509d-22c7-4a22-a47d-8c48425ef4a7,https://www.inaturalist.org/observations/13284454,Fungi,Basidiomycota,Agaricomycetes,Polyporales,Fomitopsidaceae,Fomitopsis,Fomitopsis pinicola,...,2018-06-10T02:57:15Z,CC_BY_NC_4_0,eddiecorwin,eddiecorwin,NaN,NaN,2019-03-19T19:38:37.787Z,STILLIMAGE,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS84,iNaturalist Research-grade Observations
4,1901028075,50c9509d-22c7-4a22-a47d-8c48425ef4a7,http://www.inaturalist.org/observations/1357815,Animalia,Chordata,Reptilia,Squamata,Natricidae,Thamnophis,Thamnophis sirtalis,...,2015-04-01T22:44:42Z,CC_BY_NC_4_0,colinmorita,colinmorita,NaN,NaN,2019-03-19T19:41:28.627Z,STILLIMAGE,GEODETIC_DATUM_ASSUMED_WGS84,iNaturalist Research-grade Observations
5,1901028587,50c9509d-22c7-4a22-a47d-8c48425ef4a7,https://www.inaturalist.org/observations/13966076,Animalia,Arthropoda,Insecta,Diptera,Syrphidae,Syritta,Syritta pipiens,...,2018-07-02T01:16:12Z,CC_BY_NC_4_0,Bridget Spencer,Bridget Spencer,NaN,NaN,2019-03-19T19:45:10.459Z,STILLIMAGE,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS84,iNaturalist Research-grade Observations
6,1901029035,50c9509d-22c7-4a22-a47d-8c48425ef4a7,https://www.inaturalist.org/observations/14229764,Plantae,Tracheophyta,Magnoliopsida,Rosales,Rosaceae,Rubus,Rubus laciniatus,...,2018-09-13T05:27:36Z,CC_BY_NC_4_0,sophblythe,sophblythe,NaN,NaN,2019-03-19T19:47:41.915Z,STILLIMAGE,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS84,iNaturalist Research-grade Observations
7,1901029309,50c9509d-22c7-4a22-a47d-8c48425ef4a7,https://www.inaturalist.org/observations/14373323,Plantae,Tracheophyta,Magnoliopsida,Rosales,Rosaceae,Argentina,Argentina anserina,...,2018-07-15T22:15:26Z,CC_BY_NC_4_0,citizen1,citizen1,NaN,NaN,2019-03-19T19:52:31.745Z,STILLIMAGE,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS84,iNaturalist Research-grade Observations
8,1901030041,50c9509d-22c7-4a22-a47d-8c48425ef4a7,https://www.inaturalist.org/observations/14679514,Plantae,Tracheophyta,Magnoliopsida,Malpighiales,Hypericaceae,Hypericum,Hypericum perforatum,...,2018-07-24T05:36:12Z,CC_BY_NC_4_0,citizen1,citizen1,NaN,NaN,2019-03-19T19:56:08.788Z,STILLIMAGE,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS84,iNaturalist Research-grade Observations
9,1901030500,50c9509d-22c7-4a22-a47d-8c48425ef4a7,https://www.inaturalist.org/observations/14902831,Animalia,Arthropoda,Maxillopoda,Sessilia,Balanidae,Balanus,Balanus glandula,...,2018-08-10T17:48:05Z,CC_BY_NC_4_0,voegelein,voegelein,NaN,NaN,2019-03-19T19:58:12.075Z,STILLIMAGE,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS84,iNaturalist Research-grade Observations


In [47]:
pickle_out = open("dict.pickle",'wb') 
pickle.dump(conv, pickle_out)   
pickle_out.close()

In [43]:
geo_df = gpd.GeoDataFrame(df, geometry = geometry, crs = {'init': 'epsg:4326'})

In [44]:
geo_df.to_file("leaflet/GBif.geojson", driver="GeoJSON")

DriverIOError: GeoJSON driver doesn't support creating a layer on a read-only datasource